<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## LSTM Traductor
Ejemplo basado en [LINK](https://stackabuse.com/python-for-nlp-neural-machine-translation-with-seq2seq-in-keras/)

### Datos
El objecto es utilizar datos disponibles de Anki de traducciones de texto en diferentes idiomas. Se construirá un modelo traductor seq2seq utilizando encoder-decoder.\ [LINK](https://www.manythings.org/anki/)


Replicar y extender el traductor:- Replicar el modelo en PyTorch.- Extender el entrenamiento a más datos y tamaños de secuencias mayores.- Explorar el impacto de la cantidad de neuronas en las capas recurrentes.- Mostrar 5 ejemplos de traducciones generadas.- Extras que se pueden probar: Embeddings pre-entrenados para los dos idiomas; cambiar la estrategia de generación (por ejemplo muestreo aleatorio);

# Traductor Seq2Seq con Atención Bahdanau

## Nota Importante

El siguiente código implementa un modelo Seq2Seq con mecanismo de atención Bahdanau. El modelo fue entrenado con solo 5 épocas debido a las limitaciones de tiempo y recursos de GPU, lo cual es insuficiente para obtener traducciones de alta calidad en este dataset.

## Descripción

Este proyecto implementa un traductor neuronal Seq2Seq (Sequence-to-Sequence) con LSTM y mecanismo de atención Bahdanau que traduce del inglés al español. La atención permite al modelo enfocarse en diferentes partes de la secuencia de entrada al generar cada palabra de salida.

## Arquitectura del Modelo

### Componentes Principales

- **Encoder (LSTM)**
  - Embedding dimension: 256
  - Hidden dimension: 512
  - Procesa la secuencia de entrada completa
  - Genera representaciones contextuales para cada token

- **Decoder (LSTM) con Atención Bahdanau**
  - Embedding dimension: 256
  - Hidden dimension: 512
  - Mecanismo de atención que calcula pesos dinámicos sobre las salidas del encoder
  - Concatena el contexto atencional con el embedding de entrada
  - Genera predicciones token por token

- **Mecanismo de Atención Bahdanau**
  - Calcula scores de alineamiento entre el estado oculto del decoder y todas las salidas del encoder
  - Utiliza una red feed-forward con activación tanh
  - Produce pesos de atención normalizados con softmax
  - Genera un vector de contexto ponderado

### Fórmulas de Atención
```
score = V * tanh(W1 * encoder_outputs + W2 * decoder_hidden)
attention_weights = softmax(score)
context = sum(attention_weights * encoder_outputs)
```

## Configuración del Entrenamiento

- **Dataset**: spa-eng (TensorFlow) - ~118,000 pares de oraciones
- **Épocas completadas**: 5 (de 20 planificadas)
- **Batch size**: 128
- **Máxima longitud de secuencia**: 40 tokens
- **Learning rate**: 0.001
- **Optimizador**: Adam
- **Loss**: CrossEntropyLoss (ignorando padding)
- **Teacher forcing ratio**: 0.5
- **Device**: CUDA (GPU) / CPU compatible

## Ventajas de la Atención Bahdanau

1. **Alineamiento Dinámico**: El modelo aprende automáticamente a alinear palabras entre idiomas
2. **Manejo de Secuencias Largas**: Supera el problema del "cuello de botella" del Seq2Seq básico
3. **Interpretabilidad**: Los pesos de atención muestran qué partes de la entrada son relevantes
4. **Mejor Rendimiento**: Generalmente produce traducciones más precisas que Seq2Seq sin atención

## Limitaciones Actuales

### Entrenamiento Insuficiente
- Solo 5 épocas completadas (objetivo: 20-50 épocas)
- Las traducciones pueden ser inconsistentes o de baja calidad
- El modelo aún no ha convergido completamente

### Recursos Computacionales
- Entrenamiento interrumpido por límites de tiempo de GPU
- Cada época requiere aproximadamente 15-20 minutos con GPU
- Total estimado para convergencia: 5-8 horas

## Comparación con Seq2Seq Básico

| Característica | Seq2Seq Básico | Seq2Seq + Atención |
|---------------|----------------|-------------------|
| Contexto | Vector fijo | Dinámico por paso |
| Secuencias largas | Limitado | Mejor manejo |
| Interpretabilidad | Baja | Alta (visualizar atención) |
| Parámetros | Menos | Más |
| Rendimiento | Bueno | Mejor |

## Requisitos Técnicos

- Python 3.7+
- PyTorch 1.8+
- Bibliotecas: numpy, json, zipfile, gdown


# Problemas de Entrenamiento en Colab Free

## Restricciones de Hardware

Colab Free a veces proporciona una GPU "real", pero con las siguientes limitaciones:

* **T4:** Se obtiene la mayoría del tiempo. Está **capada** (limitada en potencia, con throttling).
* **P100:** Se obtiene a veces. También con **throttling**.
* **L4:** Se obtiene muy cada tanto. Dura **poco tiempo**.
* **A100/Serias:** Nunca se obtienen en el modo gratuito.

## Restricciones por Duración de Uso

La potencia de la GPU se reduce progresivamente en función del tiempo de uso:

1.  **5–10 minutos iniciales:** La GPU opera a "full power".
2.  **Después de $\approx 10$ minutos (Uso Intensivo):** Se detecta el uso y se **baja la frecuencia** de la GPU.
3.  **Más de 1 hora de entrenamiento:** Se **baja aún más** la potencia.
4.  **2 horas o más:**
    * Se **apaga la sesión**.
    * O se deja funcionando **solo con CPU**.

In [ ]:
# ==============================
# SEQ2SEQ + ATENCIÓN (Bahdanau)
# ==============================

# 0) IMPORTS / MOUNT DRIVE
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
import zipfile
import gdown
import random
import numpy as np
import json
import tempfile
import time
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.modules.sparse import Embedding
from torch.nn.modules.rnn import LSTM, GRU
from torch.nn.modules.linear import Linear
from torch.nn.modules.dropout import Dropout
from torch.nn.modules.activation import ReLU, Tanh, Sigmoid
from torch.nn.modules.container import ModuleList

# 1) CONFIGURACIÓN RUTAS Y DEVICE
BASE_DIR = "/content/drive/MyDrive/seq2seq_bahdanau"
os.makedirs(BASE_DIR, exist_ok=True)

ZIP_FILE = "spa-eng.zip"
URL = "http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
DATA_DIR = "spa-eng"
SPA_FILE = os.path.join(DATA_DIR, "spa.txt")

CHECK_BATCH = os.path.join(BASE_DIR, "checkpoint_batch.pth")
CHECK_EPOCH = os.path.join(BASE_DIR, "checkpoint_epoch.pth")
LAST_MODEL = os.path.join(BASE_DIR, "model_last.pt")
VOCAB_FILE = os.path.join(BASE_DIR, "vocabularios.json")

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

# Parámetros
BATCH_SIZE = 128
MAX_LEN = 40
SAVE_EVERY_N_BATCHES = 20
TOTAL_EPOCHS = 20

# 2) DESCARGA Y DESCOMPRESION DATASET SI HACE FALTA
if not os.path.exists(DATA_DIR):
    if not os.path.exists(ZIP_FILE):
        print("Descargando dataset spa-eng...")
        gdown.download(URL, ZIP_FILE, quiet=False)
    print("Descomprimiendo spa-eng.zip...")
    with zipfile.ZipFile(ZIP_FILE, "r") as z:
        z.extractall(".")

# 3) LECTURA DEL DATASET
sentences_en = []
sentences_es = []

with open(SPA_FILE, encoding="utf-8") as f:
    lines = f.read().strip().split("\n")

for line in lines:
    parts = line.split("\t")
    if len(parts) >= 2:
        en, es = parts[0], parts[1]
        sentences_en.append(en.lower())
        sentences_es.append(es.lower())

print("Total de pares cargados:", len(sentences_en))

# 4) VOCABULARIOS Y ENCODING
SOS, EOS, PAD = "<sos>", "<eos>", "<pad>"

def build_vocab(sentences):
    vocab = {PAD:0, SOS:1, EOS:2}
    idx = 3
    for s in sentences:
        for w in s.split():
            if w not in vocab:
                vocab[w] = idx
                idx += 1
    inv = {i:w for w,i in vocab.items()}
    return vocab, inv

src_vocab, inv_src = build_vocab(sentences_en)
tgt_vocab, inv_tgt = build_vocab(sentences_es)

# guardar vocabularios para reuso
with open(VOCAB_FILE, "w", encoding="utf-8") as f:
    json.dump({
        "src_vocab": src_vocab,
        "tgt_vocab": tgt_vocab
    }, f, ensure_ascii=False, indent=2)

print("Vocab EN:", len(src_vocab))
print("Vocab ES:", len(tgt_vocab))

def encode(sentence, vocab):
    return [vocab[SOS]] + [vocab.get(w,0) for w in sentence.split()] + [vocab[EOS]]

# 5) DATASET + DATALOADER
class TranslationDataset(Dataset):
    def __init__(self, en, es):
        self.en = en
        self.es = es
    def __len__(self):
        return len(self.en)
    def __getitem__(self, idx):
        src = encode(self.en[idx], src_vocab)
        tgt = encode(self.es[idx], tgt_vocab)
        src = src[:MAX_LEN] + [0]*(MAX_LEN - len(src))
        tgt = tgt[:MAX_LEN] + [0]*(MAX_LEN - len(tgt))
        return torch.tensor(src, dtype=torch.long), torch.tensor(tgt, dtype=torch.long)

ds = TranslationDataset(sentences_en, sentences_es)
train_loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=False, num_workers=2, pin_memory=True)

# 6) MODELO (Bahdanau LSTM)
class BahdanauAttention(nn.Module):
    def __init__(self, hid_dim):
        super().__init__()
        self.W1 = nn.Linear(hid_dim, hid_dim)
        self.W2 = nn.Linear(hid_dim, hid_dim)
        self.V  = nn.Linear(hid_dim, 1)

    def forward(self, hidden, encoder_outputs):
        hidden_time = hidden[-1].unsqueeze(1)  # (B,1,H)
        score = torch.tanh(self.W1(encoder_outputs) + self.W2(hidden_time))
        score = self.V(score).squeeze(-1)      # (B,T)
        attn = torch.softmax(score, dim=1)     # (B,T)
        context = torch.bmm(attn.unsqueeze(1), encoder_outputs) # (B,1,H)
        return context.squeeze(1), attn

class Encoder(nn.Module):
    def __init__(self, vocab, emb=256, hid=512):
        super().__init__()
        self.embedding = nn.Embedding(vocab, emb, padding_idx=0)
        self.lstm = nn.LSTM(emb, hid, batch_first=True)
    def forward(self, src):
        emb = self.embedding(src)
        out, hidden = self.lstm(emb)
        return out, hidden

class Decoder(nn.Module):
    def __init__(self, vocab, emb=256, hid=512):
        super().__init__()
        self.embedding = nn.Embedding(vocab, emb, padding_idx=0)
        self.attn = BahdanauAttention(hid)
        self.lstm = nn.LSTM(emb + hid, hid, batch_first=True)
        self.fc = nn.Linear(hid, vocab)
    def forward(self, token, hidden, encoder_outputs):
        emb = self.embedding(token).unsqueeze(1)
        context, _ = self.attn(hidden[0], encoder_outputs)
        context = context.unsqueeze(1)
        lstm_in = torch.cat([emb, context], dim=-1)
        out, hidden = self.lstm(lstm_in, hidden)
        pred = self.fc(out.squeeze(1))
        return pred, hidden

class Seq2Seq(nn.Module):
    def __init__(self, enc, dec):
        super().__init__()
        self.enc = enc
        self.dec = dec
    def forward(self, src, tgt, teacher=0.5):
        enc_out, hidden = self.enc(src)
        B,T = tgt.shape
        outputs = torch.zeros(B, T, self.dec.fc.out_features).to(DEVICE)
        token = tgt[:,0]
        for t in range(1,T):
            out, hidden = self.dec(token, hidden, enc_out)
            outputs[:,t] = out
            token = tgt[:,t] if random.random()<teacher else out.argmax(1)
        return outputs

# 7) GUARDADO (Drive-friendly)
def _atomic_save(obj, dst_path):
    """Guardar directamente en dst_path con torch.save (Drive-friendly)."""
    try:
        torch.save(obj, dst_path)
        size = os.path.getsize(dst_path)
        print(f"Guardado OK -> {dst_path} ({size} bytes)")
        return True
    except Exception as e:
        print("Error guardando directamente en Drive:", e)
        return False

def save_batch_checkpoint(path, epoch, batch_idx, model, opt):
    payload = {
        "epoch": epoch,
        "batch": batch_idx,
        "model_state": model.state_dict(),
        "opt_state": opt.state_dict(),
        "timestamp": time.time()
    }
    _atomic_save(payload, path)

def save_epoch_checkpoint(path, epoch, model, opt):
    payload = {
        "epoch": epoch,
        "model_state": model.state_dict(),
        "opt_state": opt.state_dict(),
        "timestamp": time.time()
    }
    _atomic_save(payload, path)

def save_model_last(path, model):
    payload = {"model_state": model.state_dict(), "timestamp": time.time()}
    _atomic_save(payload, path)

def load_checkpoint_safe(path, model, opt=None):
    """
    Intenta cargar checkpoint; si está corrupto o no existe, devuelve (1,0)
    Si existe, retorna (start_epoch, start_batch) y carga estados.
    """
    if not os.path.exists(path):
        return 1, 0
    try:
        ck = torch.load(path, map_location=DEVICE)
        if "model_state" in ck:
            model.load_state_dict(ck["model_state"])
        if opt is not None and "opt_state" in ck and ck["opt_state"] is not None:
            opt.load_state_dict(ck["opt_state"])
        start_epoch = ck.get("epoch", 1)
        start_batch = ck.get("batch", 0)
        return start_epoch, start_batch
    except Exception as e:
        print("No pude cargar checkpoint (posible corrupción). Detalle:", e)
        return 1, 0

# 8) FUNCIÓN DE ENTRENAMIENTO RESILIENTE
def train_resiliente(total_epochs=TOTAL_EPOCHS):
    enc = Encoder(len(src_vocab)).to(DEVICE)
    dec = Decoder(len(tgt_vocab)).to(DEVICE)
    model = Seq2Seq(enc, dec).to(DEVICE)

    opt = torch.optim.Adam(model.parameters(), lr=0.001)
    crit = nn.CrossEntropyLoss(ignore_index=0)

    # intentar cargar checkpoint por batch
    start_epoch, start_batch = 1, 0
    if os.path.exists(CHECK_BATCH):
        print("Cargando checkpoint por batch (prioritario)...")
        se, sb = load_checkpoint_safe(CHECK_BATCH, model, opt)
        # reanudar en el siguiente batch
        start_epoch, start_batch = se, sb + 1
        print(f"Reanudando desde epoch {start_epoch}, batch {start_batch}")
    elif os.path.exists(CHECK_EPOCH):
        print("Cargando checkpoint por epoch...")
        se, sb = load_checkpoint_safe(CHECK_EPOCH, model, opt)
        # si checkpoint de epoch representa final de epoch N, reanudar en N+1
        start_epoch, start_batch = se + 1, 0
        print(f"Reanudando desde epoch {start_epoch}")

    # Entrenamiento
    for epoch in range(start_epoch, total_epochs + 1):
        model.train()
        total_loss = 0.0
        for batch_idx, (src, tgt) in enumerate(train_loader):
            # si estamos reanudando, saltamos batches ya procesados
            if epoch == start_epoch and batch_idx < start_batch:
                continue

            src = src.to(DEVICE)
            tgt = tgt.to(DEVICE)

            opt.zero_grad()
            out = model(src, tgt)
            loss = crit(out[:,1:].reshape(-1, out.shape[-1]), tgt[:,1:].reshape(-1))
            loss.backward()
            opt.step()

            total_loss += loss.item()

            # Guardar checkpoint por batch cada N batches (para no sobrecargar Drive)
            if (batch_idx % SAVE_EVERY_N_BATCHES) == 0:
                save_batch_checkpoint(CHECK_BATCH, epoch, batch_idx, model, opt)

            # imprimir progreso
            if batch_idx % 10 == 0:
                print(f"[E{epoch}/{total_epochs}] Batch {batch_idx}/{len(train_loader)}  loss={loss.item():.4f}")

        # fin epoch: guardar checkpoint por epoch y modelo "last"
        avg_loss = total_loss / len(train_loader)
        print(f"✔ Epoch {epoch} completada | loss promedio: {avg_loss:.4f}")

        # Guardados a DRIVE
        save_epoch_checkpoint(CHECK_EPOCH, epoch, model, opt)
        save_model_last(LAST_MODEL, model)

        # reset start_batch para próximas epochs
        start_batch = 0

    # al terminar, eliminar checkpoint por batch opcionalmente (ya quedó model_last)
    if os.path.exists(CHECK_BATCH):
        try:
            os.remove(CHECK_BATCH)
            print("Removed CHECK_BATCH (opcional).")
        except Exception as e:
            print("No pude borrar CHECK_BATCH:", e)
    print("Entrenamiento finalizado. Modelo guardado en:", LAST_MODEL)
    return model

# 9) FUNCIONES DE INFERENCE
torch.serialization.add_safe_globals([
    Seq2Seq, Encoder, Decoder, BahdanauAttention
])

# Clases internas de PyTorch usadas en tu modelo:
torch.serialization.add_safe_globals([
    nn.Module, nn.Sequential,
    Embedding, Linear, Dropout,
    GRU, LSTM,
    ReLU, Tanh, Sigmoid,
    ModuleList
])

def load_model_for_inference(path=LAST_MODEL):
    if not os.path.exists(path):
        raise FileNotFoundError("No existe un modelo guardado. Entrená primero.")

    # Permitimos explícitamente las clases que están en el checkpoint
    torch.serialization.add_safe_globals([Seq2Seq, Encoder, Decoder, BahdanauAttention])

    model_trained = torch.load(path, map_location=DEVICE)
    model_trained.eval()
    return model_trained

def generate(model, sentence):
    model.eval()
    src = encode(sentence, src_vocab)
    src = src[:MAX_LEN] + [0]*(MAX_LEN-len(src))
    src = torch.tensor([src]).to(DEVICE)
    with torch.no_grad():
        enc_out, hidden = model.enc(src)
        token = torch.tensor([tgt_vocab[SOS]]).to(DEVICE)
        result = []
        for _ in range(MAX_LEN):
            out, hidden = model.dec(token, hidden, enc_out)
            nxt = out.argmax(1).item()
            if nxt == tgt_vocab[EOS]:
                break
            result.append(inv_tgt.get(nxt, "<unk>"))
            token = torch.tensor([nxt]).to(DEVICE)
    return " ".join(result)

# 10) CARGAR MODELO
print("Cargando modelo completo guardado...")
model_trained = load_model_for_inference()

# Probar algunas traducciones
for i in range(5):
    s = random.choice(sentences_en)
    print("\nEN:", s)
    print("ES:", generate(model_trained, s))



Mounted at /content/drive
Device: cpu
Total de pares cargados: 118964
Vocab EN: 23851
Vocab ES: 41723
Cargando modelo completo guardado...

EN: tom invited mary to dinner.
ES: tom le a mary a mary.

EN: we love you.
ES: nos te

EN: i didn't get the point of his speech.
ES: no me la la de su

EN: i have a stomachache.
ES: tengo un un

EN: you guys need new shoes.
ES: te ves a


# Traductor Seq2Seq con Múltiples Estrategias de Decodificación

## Nota Importante

El siguiente código es una versión más simple que el anterior, pero el modelo fue entrenado con solo 10 épocas debido a las limitaciones de GPU, lo cual es insuficiente para este dataset. Las traducciones pueden estar vacías o ser de baja calidad.

## Descripción

Este proyecto implementa un traductor neuronal Seq2Seq (Sequence-to-Sequence) con LSTM que traduce del inglés al español, incluyendo 5 estrategias diferentes de decodificación.

## Arquitectura del Modelo

- **Encoder-Decoder con LSTM**
- Dimensión oculta: 256
- Capas LSTM: 1
- Dimensión embeddings: 50
- Tamaño vocabulario: 10,000 palabras (INPUT y TARGET)

## Estrategias de Decodificación Implementadas

### 1. Greedy Decoding
- Selecciona el token con mayor probabilidad en cada paso
- Determinista (siempre produce la misma salida)
- Rápido pero puede quedar atrapado en soluciones subóptimas

### 2. Sampling con Temperatura
- Muestreo estocástico de la distribución de probabilidad
- La temperatura controla la aleatoriedad (T<1: conservador, T>1: más aleatorio)
- Genera traducciones más diversas

### 3. Top-K Sampling
- Restringe el muestreo a los k tokens más probables
- k=50: considera solo las 50 palabras más probables
- Balance entre diversidad y calidad

### 4. Top-P (Nucleus) Sampling
- Muestreo dinámico basado en probabilidad acumulada
- p=0.9: selecciona el conjunto más pequeño de tokens cuya probabilidad suma ≥ 0.9
- Adapta el tamaño del vocabulario según el contexto

### 5. Beam Search
- Mantiene las top-k hipótesis en cada paso (beam_width=5)
- Explora múltiples secuencias en paralelo
- Generalmente produce mejores resultados pero es más costoso computacionalmente

## Configuración del Entrenamiento

- **Dataset**: spa-eng (TensorFlow)
- **Épocas**: 10
- **Batch size**: 64
- **Learning rate**: 1e-4
- **Optimizador**: Adam
- **Loss**: CrossEntropyLoss (ignorando padding)

## Limitaciones y Trabajo Futuro

El modelo presenta las siguientes limitaciones:

- Requiere más épocas de entrenamiento (recomendado: 30-50 épocas)
- Vocabulario limitado causa problemas con palabras poco frecuentes
- Las traducciones pueden estar vacías debido al entrenamiento insuficiente


In [1]:
!pip install torchinfo
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import pandas as pd
import re
import random
import os
import io
import zipfile
import requests
from tqdm import tqdm
from torchinfo import summary
from pathlib import Path

# ----------------------------------------------------
# 1. MONTAJE DE GOOGLE DRIVE Y SETUP
# ----------------------------------------------------

# Detección del dispositivo (prioriza GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")

# Se añade una bandera para saber si la ruta de Drive es válida.
is_colab_env = False
try:
    from google.colab import drive
    # Montar Google Drive.
    print("Montando Google Drive...")
    drive.mount('/content/drive')

    # Definir la ruta donde se guardará el modelo
    SAVE_DIR = '/content/drive/MyDrive/Modelos_Traductor'
    Path(SAVE_DIR).mkdir(parents=True, exist_ok=True)
    SAVE_PATH = os.path.join(SAVE_DIR, 'traductor_seq2seq.pt')

    is_colab_env = True
    print(f"Ruta de guardado configurada en: {SAVE_PATH}")
except ImportError:
    # Si no es Colab, guarda localmente
    SAVE_PATH = 'traductor_seq2seq.pt'
    print(f"No se detectó Colab. El modelo se guardará localmente en: {SAVE_PATH}")

# --- Hiperparámetros ajustados ---
HIDDEN_DIM = 256
NUM_LAYERS = 1
EMBEDDING_DIM = 50
MAX_VOCAB_SIZE = 10000
BATCH_SIZE = 64
NUM_EPOCHS = 10
LEARNING_RATE = 1e-4

# ----------------------------------------------------
# 2. DESCARGA Y PREPROCESAMIENTO DE DATOS
# ----------------------------------------------------

DATA_URL = "http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
DATA_PATH = "spa-eng.zip"
EXTRACT_PATH = "spa-eng"

def download_and_load_data(url, data_path, extract_path):
    """Descarga, extrae y lee el dataset."""
    if not os.path.exists(data_path):
        print(f"Descargando datos desde: {url}")
        r = requests.get(url, stream=True)
        with open(data_path, 'wb') as f:
            for chunk in tqdm(r.iter_content(chunk_size=1024)):
                if chunk:
                    f.write(chunk)

    if not os.path.exists(EXTRACT_PATH):
        with zipfile.ZipFile(data_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

    file_path = os.path.join(extract_path, 'spa-eng', 'spa.txt')
    with io.open(file_path, encoding='UTF-8') as f:
        text = f.read()

    return text

def preprocess_and_tokenize(text, max_vocab_size):
    lines = text.strip().split('\n')

    input_sentences = []
    target_sentences = []

    # Itera a través de las líneas y procesa únicamente las entradas válidas
    for line in lines:
        # Omite líneas vacías o líneas que constan únicamente de espacios en blanco
        if not line.strip():
            continue

        # Divide la línea por el carácter de tabulación
        parts = line.split('\t')

        # Comprueba si la línea tiene las dos partes esperadas (origen y destino)
        if len(parts) >= 2:
            input_sentences.append(parts[0].strip())
            # partes[1] es la oración de destino (p. ej., inglés)
            target_sentences.append(parts[1].strip())
        else:
            # OPCIONAL: Imprime la línea problemática
            print(f"Skipping malformed line: '{line}'")
            continue

    # Continua con el resto de la lógica de tokenización...
    target_sentences = [f'<sos> {s} <eos>' for s in target_sentences]

    def create_vocab(sentences, max_vocab_size):
        word_counts = {}
        for sentence in sentences:
            for word in sentence.split():
                word_counts[word] = word_counts.get(word, 0) + 1

        sorted_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
        vocab = {word: i + 2 for i, (word, count) in enumerate(sorted_words) if i < max_vocab_size - 2}

        vocab['<pad>'] = 0
        vocab['<unk>'] = 1
        idx_to_word = {v: k for k, v in vocab.items()}

        return vocab, idx_to_word

    word2idx_input, idx2word_input = create_vocab(input_sentences, max_vocab_size)
    word2idx_target, idx2word_target = create_vocab(target_sentences, max_vocab_size)

    def tokenize_sequences(sentences, word2idx):
        sequences = []
        for sentence in sentences:
            seq = [word2idx.get(word, word2idx['<unk>']) for word in sentence.split()]
            sequences.append(torch.tensor(seq, dtype=torch.int64))

        padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=word2idx['<pad>'])
        return padded_sequences

    max_input_len = max(len(s.split()) for s in input_sentences)
    max_target_len = max(len(s.split()) for s in target_sentences)

    print(f"Máxima longitud de secuencia de INPUT (EN): {max_input_len}")
    print(f"Máxima longitud de secuencia de TARGET (ES): {max_target_len}")

    encoder_input_sequences = tokenize_sequences(input_sentences, word2idx_input)
    decoder_input_sequences = tokenize_sequences(target_sentences, word2idx_target)

    target_output_sequences = decoder_input_sequences[:, 1:]

    return (encoder_input_sequences, decoder_input_sequences, target_output_sequences,
            word2idx_input, idx2word_input, word2idx_target, idx2word_target,
            max_input_len, max_target_len)

# ----------------------------------------------------
# 3. CLASES PARA EMBEDDINGS Y MODELO
# ----------------------------------------------------

class WordsEmbeddings:
    def __init__(self, n_features):
        self.N_FEATURES = n_features
    def get_words_embeddings(self, word):
        if word in ['<pad>', '<unk>', '<sos>', '<eos>']:
            return np.zeros((1, self.N_FEATURES))
        # Simula un vector válido que sería reemplazado por la carga real de GloVe
        return np.random.rand(1, self.N_FEATURES) * 0.1

class GloveEmbeddings(WordsEmbeddings):
    def __init__(self, dimension=EMBEDDING_DIM):
        super().__init__(dimension)

model_embeddings = GloveEmbeddings()

def create_embedding_matrix(word2idx, num_words, embed_dim, model_embeddings):
    """Crea la matriz de embeddings para pasar a nn.Embedding.from_pretrained."""
    embedding_matrix = np.zeros((num_words, embed_dim))
    for word, i in word2idx.items():
        if i >= num_words:
            continue
        embedding_vector = model_embeddings.get_words_embeddings(word)[0]
        if (embedding_vector is not None) and len(embedding_vector) > 0:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_layers, lstm_size, embedding_matrix=None):
        super().__init__()
        self.lstm_size = lstm_size
        self.num_layers = num_layers

        if embedding_matrix is not None:
            self.embedding = nn.Embedding.from_pretrained(
                torch.from_numpy(embedding_matrix).float(), freeze=True
            )
            embed_dim = self.embedding.embedding_dim
        else:
            self.embedding = nn.Embedding(vocab_size, embed_dim)

        self.lstm = nn.LSTM(embed_dim, self.lstm_size, self.num_layers, batch_first=True)

    def forward(self, x):
        embedded = self.embedding(x.long())
        output, hidden = self.lstm(embedded)
        return hidden

class Decoder(nn.Module):
    def __init__(self, output_dim, embed_dim, num_layers, lstm_size, embedding_matrix=None):
        super().__init__()
        self.lstm_size = lstm_size
        self.num_layers = num_layers
        self.output_dim = output_dim

        if embedding_matrix is not None:
            self.embedding = nn.Embedding.from_pretrained(
                torch.from_numpy(embedding_matrix).float(), freeze=True
            )
            embed_dim = self.embedding.embedding_dim
        else:
            self.embedding = nn.Embedding(output_dim, embed_dim)

        self.lstm = nn.LSTM(embed_dim, self.lstm_size, self.num_layers, batch_first=True)
        self.out = nn.Linear(self.lstm_size, output_dim)

    def forward(self, x, hidden):
        embedded = self.embedding(x.long())
        output, hidden = self.lstm(embedded, hidden)
        prediction = self.out(output.squeeze(1))
        return prediction, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, target_len):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.target_len = target_len

    def forward(self, encoder_input, decoder_input_start):
        batch_size = decoder_input_start.shape[0]
        vocab_size = self.decoder.output_dim
        outputs = torch.zeros(batch_size, self.target_len, vocab_size).to(device)
        prev_state = self.encoder(encoder_input)

        for t in range(self.target_len):
            input_t = decoder_input_start[:, t:t+1]
            output, prev_state = self.decoder(input_t, prev_state)
            outputs[:, t, :] = output

        return outputs

class MTDataset(Dataset):
    def __init__(self, input_data, decoder_input, target_output):
        self.input_data = input_data
        self.decoder_input = decoder_input
        self.target_output = target_output
    def __len__(self):
        return len(self.input_data)
    def __getitem__(self, idx):
        return self.input_data[idx], self.decoder_input[idx], self.target_output[idx]

def train(model, dataloader, optimizer, criterion, clip=1):
    """Bucle de entrenamiento de una época."""
    model.train()
    epoch_loss = 0

    for i, (input_seq, decoder_input_seq, target_output_seq) in enumerate(dataloader):
        input_seq = input_seq.to(device)
        decoder_input_seq = decoder_input_seq.to(device)
        target_output_seq = target_output_seq.to(device)

        optimizer.zero_grad()
        output = model(input_seq, decoder_input_seq)

        output_dim = output.shape[-1]
        output = output.reshape(-1, output_dim)
        target_output_seq = target_output_seq.reshape(-1)

        loss = criterion(output, target_output_seq)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

# ----------------------------------------------------
# 4. INFERENCIA Y GENERACIÓN
# ----------------------------------------------------

def prepare_input_translation(sentence, word2idx, max_input_len):
    """Tokeniza y prepara un tensor de entrada para la traducción."""
    sentence = sentence.lower()
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)

    tokens = [word2idx.get(word, word2idx['<unk>']) for word in sentence.split()]

    padded_tokens = np.zeros(max_input_len, dtype=np.int32)
    padded_tokens[-len(tokens):] = tokens

    input_tensor = torch.from_numpy(padded_tokens).unsqueeze(0).to(device)
    return input_tensor

def translate_sentence(encoder_sequence_test_tensor, model, idx2word_target, word2idx_target, max_len, strategy='greedy'):
    """Traduce una secuencia de entrada (Greedy o Sample)."""
    model.eval()

    sos_idx = word2idx_target['<sos>']
    eos_idx = word2idx_target['<eos>']

    with torch.no_grad():
        prev_state = model.encoder(encoder_sequence_test_tensor)

    target_seq = torch.ones(1, 1).fill_(sos_idx).int().to(device)
    translation = []

    for t in range(max_len):
        with torch.no_grad():
            output_logits, prev_state = model.decoder(target_seq, prev_state)

        probabilities = F.softmax(output_logits.squeeze(0), dim=-1)

        if strategy == 'greedy':
            next_token_idx = probabilities.argmax().item()
        elif strategy == 'sample':
            next_token_idx = torch.multinomial(probabilities, num_samples=1).item()
        else:
            raise ValueError("Estrategia no reconocida. Use 'greedy' o 'sample'.")

        if next_token_idx == eos_idx:
            break

        word = idx2word_target.get(next_token_idx, '<unk>')
        if word not in ['<pad>', '<sos>', '<eos>', '<unk>']:
            translation.append(word)

        target_seq = torch.tensor([[next_token_idx]], dtype=torch.int64).to(device)

    model.train()
    return ' '.join(translation)

# ----------------------------------------------------
# 5. EJECUCIÓN PRINCIPAL Y CONFIGURACIÓN DEL MODELO
# ----------------------------------------------------

# 1. Carga y preprocesamiento
raw_text = download_and_load_data(DATA_URL, DATA_PATH, EXTRACT_PATH)
(encoder_input_sequences, decoder_input_sequences, target_output_sequences,
 word2idx_input, idx2word_input, word2idx_target, idx2word_target,
 max_input_len, max_target_len) = preprocess_and_tokenize(raw_text, MAX_VOCAB_SIZE)

# 2. Preparación de Embeddings
num_words_input = len(word2idx_input)
num_words_target = len(word2idx_target)

embedding_matrix_input = create_embedding_matrix(word2idx_input, num_words_input, EMBEDDING_DIM, model_embeddings)
embedding_matrix_output = create_embedding_matrix(word2idx_target, num_words_target, EMBEDDING_DIM, model_embeddings)

# 3. Inicialización del Modelo
encoder = Encoder(vocab_size=num_words_input, embed_dim=EMBEDDING_DIM, num_layers=NUM_LAYERS,
                  lstm_size=HIDDEN_DIM, embedding_matrix=embedding_matrix_input)
decoder = Decoder(output_dim=num_words_target, embed_dim=EMBEDDING_DIM, num_layers=NUM_LAYERS,
                  lstm_size=HIDDEN_DIM, embedding_matrix=embedding_matrix_output)

model = Seq2Seq(encoder, decoder, target_len=target_output_sequences.shape[1]).to(device)

print("-" * 50)
print(f"Modelo Seq2Seq con HIDDEN_DIM={HIDDEN_DIM}")
print("-" * 50)

# 4. DataLoaders y Optimizador
dataset = MTDataset(encoder_input_sequences, decoder_input_sequences, target_output_sequences)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

criterion = nn.CrossEntropyLoss(ignore_index=word2idx_target['<pad>']).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# ----------------------------------------------------
# 6. CARGAR O ENTRENAR Y GUARDAR
# ----------------------------------------------------

if os.path.exists(SAVE_PATH):
    print(f"\n--- CARGANDO MODELO ENTRENADO ---")
    # Cargar el modelo asegurando que se mapee al dispositivo correcto (CPU o CUDA)
    model.load_state_dict(torch.load(SAVE_PATH, map_location=device))
    print("¡Modelo cargado con éxito desde Google Drive! Saltando entrenamiento.")
else:
    print(f"\n--- INICIO DEL ENTRENAMIENTO ---")
    print(f"Modelo no encontrado en {SAVE_PATH}. Iniciando entrenamiento desde cero.")

    for epoch in range(NUM_EPOCHS):
        train_loss = train(model, dataloader, optimizer, criterion)
        print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f}')

    # Guardar el modelo recién entrenado
    print(f"\nGuardando el estado del modelo entrenado en: {SAVE_PATH}...")
    torch.save(model.state_dict(), SAVE_PATH)
    print("¡Modelo guardado con éxito en Google Drive!")

# ----------------------------------------------------
# 7. EJEMPLOS DE TRADUCCIÓN CON TODAS LAS ESTRATEGIAS
# ----------------------------------------------------
def translate_sentence(encoder_sequence_test_tensor, model, idx2word_target, word2idx_target, max_len, strategy='greedy', **kwargs):
    """
    Traduce una secuencia de entrada usando diferentes estrategias de decodificación.
    """
    model.eval()

    sos_idx = word2idx_target['<sos>']
    eos_idx = word2idx_target['<eos>']

    # Beam Search (requiere lógica diferente)
    if strategy == 'beam_search':
        beam_width = kwargs.get('beam_width', 5)

        with torch.no_grad():
            encoder_state = model.encoder(encoder_sequence_test_tensor)

        beams = [(torch.ones(1, 1).fill_(sos_idx).int().to(device), 0.0, encoder_state)]
        completed_beams = []

        for t in range(max_len):
            candidates = []

            for seq, score, state in beams:
                if seq[0, -1].item() == eos_idx:
                    completed_beams.append((seq, score))
                    continue

                with torch.no_grad():
                    output_logits, new_state = model.decoder(seq[:, -1:], state)

                log_probs = F.log_softmax(output_logits.squeeze(0), dim=-1)
                top_log_probs, top_indices = torch.topk(log_probs, beam_width)

                for i in range(beam_width):
                    next_token = top_indices[i].item()
                    next_score = score + top_log_probs[i].item()
                    next_seq = torch.cat([seq, torch.tensor([[next_token]], dtype=torch.int64).to(device)], dim=1)
                    candidates.append((next_seq, next_score, new_state))

            if not candidates:
                break

            beams = sorted(candidates, key=lambda x: x[1], reverse=True)[:beam_width]

        completed_beams.extend([(seq, score) for seq, score, _ in beams])

        if not completed_beams:
            model.train()
            return ""

        best_seq, _ = max(completed_beams, key=lambda x: x[1])

        translation = []
        for token_idx in best_seq[0, 1:].tolist():
            if token_idx == eos_idx:
                break
            word = idx2word_target.get(token_idx, '<unk>')
            if word not in ['<pad>', '<sos>', '<eos>', '<unk>']:
                translation.append(word)

        model.train()
        return ' '.join(translation)

    # Estrategias paso a paso (greedy, sample, top_k, top_p)
    with torch.no_grad():
        prev_state = model.encoder(encoder_sequence_test_tensor)

    target_seq = torch.ones(1, 1).fill_(sos_idx).int().to(device)
    translation = []

    for t in range(max_len):
        with torch.no_grad():
            output_logits, prev_state = model.decoder(target_seq, prev_state)

        probabilities = F.softmax(output_logits.squeeze(0), dim=-1)

        if strategy == 'greedy':
            next_token_idx = probabilities.argmax().item()

        elif strategy == 'sample':
            temperature = kwargs.get('temperature', 1.0)
            if temperature != 1.0:
                probabilities = F.softmax(output_logits.squeeze(0) / temperature, dim=-1)
            next_token_idx = torch.multinomial(probabilities, num_samples=1).item()

        elif strategy == 'top_k':
            k = kwargs.get('k', 50)
            top_k_probs, top_k_indices = torch.topk(probabilities, k)
            top_k_probs = top_k_probs / top_k_probs.sum()
            next_token_idx = top_k_indices[torch.multinomial(top_k_probs, num_samples=1)].item()

        elif strategy == 'top_p':
            p = kwargs.get('p', 0.9)
            sorted_probs, sorted_indices = torch.sort(probabilities, descending=True)
            cumulative_probs = torch.cumsum(sorted_probs, dim=-1)

            sorted_indices_to_remove = cumulative_probs > p
            sorted_indices_to_remove[1:] = sorted_indices_to_remove[:-1].clone()
            sorted_indices_to_remove[0] = False

            filtered_probs = sorted_probs.clone()
            filtered_probs[sorted_indices_to_remove] = 0
            filtered_probs = filtered_probs / filtered_probs.sum()

            next_token_idx = sorted_indices[torch.multinomial(filtered_probs, num_samples=1)].item()

        else:
            raise ValueError(f"Estrategia no reconocida: {strategy}")

        if next_token_idx == eos_idx:
            break

        word = idx2word_target.get(next_token_idx, '<unk>')
        if word not in ['<pad>', '<sos>', '<eos>', '<unk>']:
            translation.append(word)

        target_seq = torch.tensor([[next_token_idx]], dtype=torch.int64).to(device)

    model.train()
    return ' '.join(translation)

print("\n" + "=" * 50)
print("Generación de Ejemplos con Diferentes Estrategias de Decodificación")
print("=" * 50)

test_sentences = [
    "Tom is here.",
    "Go home.",
    "I am Tom.",
    "Come here.",
    "Be quiet."
]

for i, input_sentence in enumerate(test_sentences):
    input_tensor = prepare_input_translation(input_sentence, word2idx_input, max_input_len)

    print(f"\n[{i+1}] INPUT (EN): {input_sentence}")

    # Greedy Decoding
    greedy_translation = translate_sentence(
        input_tensor, model, idx2word_target, word2idx_target, max_target_len, strategy='greedy')
    print(f"    -> Greedy:           {greedy_translation}")

    # Muestreo Aleatorio (con temperatura)
    sample_translation = translate_sentence(
        input_tensor, model, idx2word_target, word2idx_target, max_target_len,
        strategy='sample', temperature=0.8)
    print(f"    -> Sample (T=0.8):   {sample_translation}")

    # Top-k Sampling
    topk_translation = translate_sentence(
        input_tensor, model, idx2word_target, word2idx_target, max_target_len,
        strategy='top_k', k=50)
    print(f"    -> Top-k (k=50):     {topk_translation}")

    # Top-p (Nucleus) Sampling
    topp_translation = translate_sentence(
        input_tensor, model, idx2word_target, word2idx_target, max_target_len,
        strategy='top_p', p=0.9)
    print(f"    -> Top-p (p=0.9):    {topp_translation}")

    # Beam Search
    beam_translation = translate_sentence(
        input_tensor, model, idx2word_target, word2idx_target, max_target_len,
        strategy='beam_search', beam_width=5)
    print(f"    -> Beam Search (w=5): {beam_translation}")


Usando dispositivo: cpu
Montando Google Drive...
Mounted at /content/drive
Ruta de guardado configurada en: /content/drive/MyDrive/Modelos_Traductor/traductor_seq2seq.pt
Descargando datos desde: http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip


2577it [00:00, 51242.68it/s]


Máxima longitud de secuencia de INPUT (EN): 47
Máxima longitud de secuencia de TARGET (ES): 51
--------------------------------------------------
Modelo Seq2Seq con HIDDEN_DIM=256
--------------------------------------------------

--- CARGANDO MODELO ENTRENADO ---
¡Modelo cargado con éxito desde Google Drive! Saltando entrenamiento.

Generación de Ejemplos con Diferentes Estrategias de Decodificación

[1] INPUT (EN): Tom is here.
    -> Greedy:           
    -> Sample (T=0.8):   
    -> Top-k (k=50):     
    -> Top-p (p=0.9):    
    -> Beam Search (w=5): 

[2] INPUT (EN): Go home.
    -> Greedy:           
    -> Sample (T=0.8):   Mire bastante
    -> Top-k (k=50):     Eso
    -> Top-p (p=0.9):    Tom bien.
    -> Beam Search (w=5): 

[3] INPUT (EN): I am Tom.
    -> Greedy:           
    -> Sample (T=0.8):   
    -> Top-k (k=50):     es se
    -> Top-p (p=0.9):    
    -> Beam Search (w=5): 

[4] INPUT (EN): Come here.
    -> Greedy:           
    -> Sample (T=0.8):   
    -> To